**Import required libraries and scripts**

In [2]:
import os
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.performance_calculation import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *


[08:39:19] Initializing Normalizer
/home/mario/anaconda3/envs/wocondock/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.
/home/mario/anaconda3/envs/wocondock/lib/python3.8/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  class NCDFPicklable(scipy.io.netcdf.netcdf_file):


In [ ]:

for dir in os.listdir('/media/mario/T7/FINISHED/FINISHED/'):
    print(dir)
    software = '/home/mario/DockM8/software'
    protein_file = f'/media/mario/T7/FINISHED/FINISHED/{dir}/receptor_protoss_prepared.pdb'
    ref_file = f'/media/mario/T7/FINISHED/FINISHED/{dir}/crystal_ligand_protoss.sdf'
    pocket = 'reference'
    protonation = 'pkasolver'
    docking_library = f'/media/mario/T7/FINISHED/FINISHED/{dir}/merged_actives_decoys.sdf'
    docking_programs = ['GNINA', 'SMINA', 'PLANTS']
    clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
    clustering_method = 'KMedoids'
    rescoring= ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'SCORCH', 'RTMScore', 'vinardo']
    id_column = 'ID'
    n_poses = 10
    exhaustiveness = 8
    parallel = 1
    ncpus = int(os.cpu_count()/2)
    #Create a temporary folder for all further calculations
    w_dir = os.path.dirname(protein_file)
    print('The working directory has been set to:', w_dir)
    create_temp_folder(w_dir+'/temp')
    
    try:   
        apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)
    except Exception as e:
        printlog(f'Failed for {dir}')
        print(e)
        


In [3]:
def rename_columns_in_csv(root_dir, target_file, col_rename_dict):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_file:
                filepath = os.path.join(dirpath, filename)

                # Read the CSV file
                df = pd.read_csv(filepath)

                # Rename columns according to the dictionary
                df.rename(columns=col_rename_dict, inplace=True)

                # Write the modified CSV file back to the same location
                df.to_csv(filepath, index=False)
                print(f"Columns renamed in: {filepath}")

root_directory = "/media/mario/T7/FINISHED/FINISHED/"
target_csv_file = "allposes_rescored.csv"
column_rename_dict = {
    "AD4_Affinity": "AD4",
    "Vinardo_Affinity": "Vinardo",
    "GNINA_Affinity": "GNINA",
    "GNINA_CNN_Score": "CNN-Score",
    "GNINA_CNN_Affinity": "CNN-Affinity",
    'LinF9_Affinity':'LinF9',
    'SCORCH_pose_score':'SCORCH',
    # Add more column names to be renamed here
}
rename_columns_in_csv(root_directory, target_csv_file, column_rename_dict)

Columns renamed in: /media/mario/T7/FINISHED/FINISHED/abl1/temp/rescoring_3DScore_clustered/allposes_rescored.csv
Columns renamed in: /media/mario/T7/FINISHED/FINISHED/abl1/temp/rescoring_bestpose_clustered/allposes_rescored.csv
Columns renamed in: /media/mario/T7/FINISHED/FINISHED/abl1/temp/rescoring_bestpose_GNINA_clustered/allposes_rescored.csv
Columns renamed in: /media/mario/T7/FINISHED/FINISHED/abl1/temp/rescoring_bestpose_PLANTS_clustered/allposes_rescored.csv
Columns renamed in: /media/mario/T7/FINISHED/FINISHED/abl1/temp/rescoring_bestpose_SMINA_clustered/allposes_rescored.csv
Columns renamed in: /media/mario/T7/FINISHED/FINISHED/abl1/temp/rescoring_espsim_clustered/allposes_rescored.csv
Columns renamed in: /media/mario/T7/FINISHED/FINISHED/abl1/temp/rescoring_RMSD_clustered/allposes_rescored.csv
Columns renamed in: /media/mario/T7/FINISHED/FINISHED/abl1/temp/rescoring_spyRMSD_clustered/allposes_rescored.csv
Columns renamed in: /media/mario/T7/FINISHED/FINISHED/ace/temp/rescor

In [12]:
import os
import pandas as pd

def rename_and_merge_csv_files(root_dir, target_file, output_file):
    merged_df = None

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_file:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath, index_col=0)
                dirname = os.path.basename(dirpath)
                print(dirpath.replace('/media/mario/T7/FINISHED/FINISHED/', '').replace('/temp/consensus', ''))
                df.rename(columns={"EF1%": dirpath.replace('/media/mario/T7/FINISHED/FINISHED/', '').replace('/temp/consensus', '')}, inplace=True)
                df.drop(columns='EF10%', inplace=True)
                if merged_df is None:
                    merged_df = df
                else:
                    merged_df = pd.merge(merged_df, df, on=["Scoring Function", "Clustering Metric"])

    if merged_df is not None:
        # Adding the average column
        columns_to_exclude = ['method_name', 'selected_columns', 'clustering_metric']
        numeric_columns = [col for col in merged_df.columns if col not in columns_to_exclude]
        merged_df['Average'] = merged_df[numeric_columns].mean(axis=1)
        merged_df.to_csv(output_file)
        print(f"Merged CSV file saved to: {output_file}")


root_directory = "/media/mario/T7/FINISHED/FINISHED/"
target_csv_file = "EF_single_functions.csv"
output_csv_file = "merged_output1.csv"

rename_and_merge_csv_files(root_directory, target_csv_file, output_csv_file)


abl1
ace
aldr
andr
cdk2
cox1
dhi1
dyr
esr1
fa7
fabp4
gria2
hdac2
hivrt
hmdh
hs90a
hxk4
igf1r
kith
kpcb
lfa1
lkha4
nram
prgr
ptn1
pyrd
reni
urok
xiap
Merged CSV file saved to: merged_output1.csv


In [11]:
import os
import pandas as pd

def rename_and_merge_csv_files(root_dir, target_file, output_file):
    merged_df = None

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_file:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath, index_col=0)
                dirname = os.path.basename(dirpath)
                df.rename(columns={"EF1%": dirpath.replace('/media/mario/T7/FINISHED/FINISHED/', '').replace('/temp/consensus', '')}, inplace=True)
                if merged_df is None:
                    merged_df = df
                else:
                    merged_df = pd.merge(merged_df, df, on=["clustering_method", "selected_columns", 'method_name'])

    if merged_df is not None:
        # Adding the average column
        columns_to_exclude = ['method_name', 'selected_columns', 'clustering_metric']
        numeric_columns = [col for col in merged_df.columns if col not in columns_to_exclude]
        merged_df['Average'] = merged_df[numeric_columns].mean(axis=1)
        print(merged_df.head())
        merged_df.to_csv(output_file)
        print(f"Merged CSV file saved to: {output_file}")

root_directory = "/media/mario/T7/FINISHED/FINISHED/"
target_csv_file = "consensus_summary.csv"
output_csv_file = "merged_output_consensus1.csv"

rename_and_merge_csv_files(root_directory, target_csv_file, output_csv_file)


                  method_name selected_columns   abl1    ace   aldr   andr  \
clustering_method                                                            
RMSD                  method1  CHEMPLPFScoreVS  23.57  23.42  24.03  16.04   
RMSD                  method2  CHEMPLPFScoreVS  27.96  23.77  15.17  24.99   
RMSD                  method3  CHEMPLPFScoreVS  19.18  20.93  10.75  17.90   
RMSD                  method4  CHEMPLPFScoreVS  17.54  20.58  10.75  13.80   
RMSD                  method5  CHEMPLPFScoreVS  31.24  21.64  22.13  30.21   

                   cdk2   cox1   dhi1    dyr  ...   lfa1  lkha4   nram   prgr  \
clustering_method                             ...                               
RMSD               6.32  10.80  10.92   6.05  ...   2.91  34.60  11.21  20.89   
RMSD               8.01   6.68  15.47  10.38  ...  16.73  32.84  11.21  19.18   
RMSD               5.27   4.11   9.40   6.05  ...   9.45  28.15   5.09  14.38   
RMSD               4.85   5.14   8.49   6.49  ..